In [1]:
import agentpy as ap

# Visualization
import matplotlib.pyplot as plt

import seaborn as sns
import IPython
import random
import time
import pandas as pd

In [2]:
casillas = 1
valid_moves = []
for i in range(casillas+1):
    for j in range(casillas+1):
        valid_moves.append((-i,-j))
        valid_moves.append((i, -j))
        valid_moves.append((-i,j))
        valid_moves.append((i,j))

# Eliminar valores repetidos
valid_moves = [*set(valid_moves)]

In [3]:
class CleaningRooms(ap.Model):
    
    def setup(self):

        # Create agents (trees)
        n_habitaciones = int(self.p['Rooms uncleaned'] * (self.p.Height * self.p.Length))
        
        #trees = self.agents = self.add_agents(n_trees)
        habitaciones = self.agents = ap.AgentList(self, n_habitaciones)

        ## Create agents (cleaning machine)
        n_cleaners = int(self.p['Cleaners'])
        cleaners = self.other_agents = ap.AgentList(self, n_cleaners)
        
        # Create grid (cuarto)
        #self.espacio = ap.Grid(self, [self.p.size]*2, track_empty=True)
        self.espacio = ap.Grid(self, (self.p.Height, self.p.Length), track_empty=True)
        self.espacio.add_agents(habitaciones, random=True, empty=True)

        tuple_positions = [(1,1) for _ in range(len(self.other_agents)) ]
        self.espacio.add_agents(cleaners, positions=tuple_positions,empty=True )
        # Initiate a dynamic variable for all trees
        # Condition 0: Uncleaned, 1: Cleaned, 2: Cleaning, 3: Agents machines
        self.agents.condition = 0
        self.other_agents.condition = 3 #3

    def step(self):


        # Select cleaning rooms
        cleaning_rooms = self.other_agents.select(self.agents.condition == 1)
        
        global total_steps
        #for hab in cleaning_rooms:
        total_steps += 1
        
        for ag in self.other_agents:
            # Clean rooms
            for neighbor in self.espacio.neighbors(ag,0):    
                # Change uncleaned condition to cleaning
                if neighbor.condition == 0:
                    neighbor.condition = 2 #2 
                    break
                    
                # Once room is cleaned move to another position
                elif neighbor.condition == 2: #2
                    neighbor.condition = 1
                 
            else:
                random_move = random.choice(valid_moves)
                #for neighbor in self.espacio.neighbors(hab):
                self.espacio.move_by(ag, random_move)
                global moves_room 
                moves_room += 1
        

    def end(self):

        # Document a measure at the end of the simulation
        cleaned_rooms = len(self.agents.select(self.agents.condition == 1))
        self.report('Percentage of cleaned rooms',
                    cleaned_rooms / len(self.agents))
        self.report('cleaned_rooms', cleaned_rooms)
        


In [9]:
# Define parameters

parameters = {
    'Rooms uncleaned': 1, #  0.9, # Percentage of rooms uncleaned
    'Cleaners': 1, # Machine cleaners
    'Length': 10, # Length of the grid
    'Height': 30, # Height of the grid
    'steps': 100 # Max steps
}

In [13]:
# Create single-run animation with custom colors

moves_room = 0
total_steps = 0
def animation_plot(model, ax):
    attr_grid = model.espacio.attr_grid('condition')
    color_dict = {0:'#986F53', 1:'#FFFFFF', 2:'#CAFFFD', 3:'#0060ff' , None:'#D4D4D4'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    
    ax.set_title(
                fr"$\bfSimulation\ of\ agents\ cleaning\ rooms $"
                 "\nRooms left: "
                 f"{len(model.agents.select(model.agents.condition == 0))}\n"
                 f"Total agents: {parameters['Cleaners']}\n"
                 f"Total steps: {total_steps}\n"
                 f"Percentage rooms cleaned: % "
                 f"{int(((len(model.agents.select(model.agents.condition==1)))/(len(model.agents)))*100)}\n"
                 
        
                 f'Moves realized by all agents: {moves_room}'
                )

px = 1/plt.rcParams['figure.dpi']
plt.rcParams.update({'font.size': 12})


x = parameters['Length']
y = parameters['Height']
             
     
if x <30:
    sizeX = 30
else:
    sizeX = x
    
if y < 40:
    sizeY = 65
elif y <= 10:
    
    sizeY = 25
else:
    sizeY = y


fig, ax = plt.subplots(figsize=(sizeX*12*px, sizeY *14*px))
model = CleaningRooms(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
ax.axis('scaled') 

IPython.display.HTML(animation.to_jshtml(fps=15))